## 0. 필요한 라이브러리 설치

In [1]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents

Note: you may need to restart the kernel to use updated packages.


## 1. 환경 변수 설정

In [2]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")

Search Endpoint: https://ai-search-commercial-1023.search.windows.net
Search Index: products-index


## 2. Augmented 데이터 로드

In [3]:
import json
from pathlib import Path

##############################################
# 2. Augmented 데이터 로드
##############################################
data_file = Path("data/sample_products_augmented.json")

if not data_file.exists():
    print(f"❌ 파일을 찾을 수 없습니다: {data_file}")
    print("먼저 07_image_to_product_info.ipynb를 실행하여 augmented 데이터를 생성하세요.")
else:
    with open(data_file, "r", encoding="utf-8") as f:
        products = json.load(f)
    
    print(f"✅ 총 {len(products)}개의 augmented 상품 데이터를 로드했습니다.")
    print(f"\n첫 번째 상품 예시:")
    print(json.dumps(products[0], indent=2, ensure_ascii=False))

✅ 총 30개의 augmented 상품 데이터를 로드했습니다.

첫 번째 상품 예시:
{
  "id": "PROD001",
  "name": "클래식 화이트 티셔츠",
  "brand": "베이직코튼",
  "description": "100% 순면으로 제작된 기본 화이트 티셔츠입니다. 데일리룩으로 완벽하며 편안한 착용감을 제공합니다.",
  "imageUrl": "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?w=800&q=80",
  "price": 29000,
  "imageCaption": "흰색 티셔츠를 입은 사람.",
  "imageDescription": "이미지에는 심플하고 기본적인 흰색 반팔 티셔츠를 입은 사람이 등장하며, 배경은 식물이 있는 실내 공간입니다.",
  "imageTags": [
    "티셔츠",
    "흰색",
    "반팔",
    "심플",
    "식물",
    "실내"
  ]
}


In [4]:
from IPython.display import display, HTML

##############################################
# 2-1. Augmented 데이터 시각화
##############################################
print(f"📦 Augmented 상품 목록 ({len(products)}개)")
print("="*80)

# HTML로 이미지와 정보를 보기 좋게 표시
html_content = '<div style="display: flex; flex-wrap: wrap; gap: 20px;">'

for idx, product in enumerate(products[:3], 1):
    image_tags = ', '.join(product.get('imageTags', [])[:3])
    html_content += f'''
    <div style="border: 1px solid #ddd; border-radius: 8px; padding: 15px; width: 300px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        <div style="font-weight: bold; font-size: 14px; color: #333; margin-bottom: 10px;">
            {idx}. {product["name"]}
        </div>
        <img src="{product["imageUrl"]}" style="width: 100%; height: 200px; object-fit: cover; border-radius: 4px; margin-bottom: 10px;" />
        <div style="font-size: 11px; color: #0066cc; margin-bottom: 5px; font-style: italic;">
            💬 {product.get("imageCaption", "N/A")}
        </div>
        <div style="font-size: 11px; color: #666; margin-bottom: 5px;">
            <strong>브랜드:</strong> {product["brand"]} | <strong>가격:</strong> {product["price"]:,}원
        </div>
        <div style="font-size: 10px; color: #888; line-height: 1.3; margin-bottom: 5px;">
            {product.get("imageDescription", "")[:80]}...
        </div>
        <div style="font-size: 10px; color: #28a745; background: #e8f5e9; padding: 4px; border-radius: 3px;">
            🏷️ {image_tags}
        </div>
    </div>
    '''

html_content += '</div>'

display(HTML(html_content))

📦 Augmented 상품 목록 (30개)


## 3. 클라이언트 초기화

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
import os, shutil

##############################################
# 3-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))

##############################################
# 3-1. Azure 인증 설정
##############################################
credential = DefaultAzureCredential()

##############################################
# 3-2. AI Search 클라이언트 생성
##############################################
search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

print("✅ Search 클라이언트가 초기화되었습니다.")

az which: /opt/homebrew/bin/az
✅ Search 클라이언트가 초기화되었습니다.


## 4. 이미지 필드 업데이트용 문서 준비

In [6]:
##############################################
# 4. 이미지 관련 필드만 포함한 문서 준비
##############################################

documents = []

print(f"🚀 {len(products)}개의 제품 이미지 필드를 준비합니다...\n")

for product in products:
    # 이미지 관련 필드만 포함한 문서 구성
    document = {
        "id": product["id"],
        "imageCaption": product.get("imageCaption", ""),
        "imageDescription": product.get("imageDescription", ""),
        "imageTags": product.get("imageTags", [])
    }
    documents.append(document)

print(f"✅ 총 {len(documents)}개의 문서 업데이트 준비 완료")
print(f"\n첫 번째 문서 예시:")
print(json.dumps(documents[0], indent=2, ensure_ascii=False))

🚀 30개의 제품 이미지 필드를 준비합니다...

✅ 총 30개의 문서 업데이트 준비 완료

첫 번째 문서 예시:
{
  "id": "PROD001",
  "imageCaption": "흰색 티셔츠를 입은 사람.",
  "imageDescription": "이미지에는 심플하고 기본적인 흰색 반팔 티셔츠를 입은 사람이 등장하며, 배경은 식물이 있는 실내 공간입니다.",
  "imageTags": [
    "티셔츠",
    "흰색",
    "반팔",
    "심플",
    "식물",
    "실내"
  ]
}


## 5. 인덱스 업데이트

In [7]:
import time

##############################################
# 5-1. 기존 문서 확인
##############################################
try:
    existing_docs = list(search_client.search(search_text="*", select=["id"], top=1000))
    existing_ids = set([doc["id"] for doc in existing_docs])
    print(f"📊 인덱스에 {len(existing_ids)}개의 기존 문서가 있습니다.")
except Exception as e:
    print(f"⚠️ 기존 문서 조회 중 오류 (새로운 인덱스일 수 있음): {e}")
    existing_ids = set()

##############################################
# 5-2. 이미지 필드만 업데이트 (Merge)
##############################################
# merge_documents: 기존 문서의 특정 필드만 업데이트 (다른 필드는 유지)
result = search_client.merge_documents(documents=documents)

success_count = sum([1 for r in result if r.succeeded])
fail_count = sum([1 for r in result if not r.succeeded])

print(f"\n✅ 문서 업로드 완료")
print(f"   - 성공: {success_count}개")
print(f"   - 실패: {fail_count}개")

# 실패한 문서가 있으면 상세 정보 출력
if fail_count > 0:
    print("\n❌ 실패한 문서 상세:")
    for r in result:
        if not r.succeeded:
            print(f"   - ID: {r.key}, 오류: {r.error_message}")
            
time.sleep(2)  # 잠시 대기

📊 인덱스에 30개의 기존 문서가 있습니다.

✅ 문서 업로드 완료
   - 성공: 30개
   - 실패: 0개


## 6. 업로드 확인

In [8]:
##############################################
# 6-1. 업로드된 문서 확인
##############################################
results = search_client.search(
    search_text="*", 
    select=["id", "name", "brand", "price", "imageCaption", "imageDescription", "imageTags"],
    top=3
)

print("📋 업로드된 상품 샘플 (3개):")
print("="*80)

for idx, result in enumerate(results, 1):
    print(f"\n{idx}. ID: {result['id']}")
    print(f"   이름: {result['name']}")
    print(f"   브랜드: {result['brand']}")
    print(f"   가격: {result['price']:,.0f}원")
    print(f"   이미지 캡션: {result.get('imageCaption', 'N/A')}")
    print(f"   이미지 설명: {result.get('imageDescription', 'N/A')}")
    print(f"   이미지 태그: {', '.join(result.get('imageTags', []))}")

print("\n" + "="*80)

📋 업로드된 상품 샘플 (3개):

1. ID: PROD018
   이름: 스니커즈
   브랜드: 어바샬
   가격: 69,000원
   이미지 캡션: 보라색 스니커즈가 노란 배경에서 공중에 떠 있는 모습.
   이미지 설명: 이미지에는 노란 배경 앞에 보라색 스니커즈가 한 켤레 공중에 떠 있는 상태로 나타나 있습니다. 신발은 심플한 디자인으로 흰색 끈과 흰색 밑창을 특징으로 하고, 옆면에 곡선형 흰색 줄무늬가 추가로 장식되어 있습니다.
   이미지 태그: 스니커즈, 신발, 패션, 보라색, 디자인, 심플, 밑창, 끈

2. ID: PROD028
   이름: 골프 폴로 셔츠
   브랜드: 스포츠클럽
   가격: 69,000원
   이미지 캡션: 다양한 컬러가 조화를 이루는 타이다이 셔츠 컬렉션의 이미지.
   이미지 설명: 이미지는 옷걸이에 걸려 있는 여러 색상의 타이다이 셔츠를 보여줍니다. 각 셔츠는 부드럽고 밝은 색조로 되어 있으며, 색의 혼합이 독특합니다. 청색, 녹색, 오렌지색 등의 다양한 컬러 조합이 돋보이며, 긴 소매와 단추 디테일이 있어 세련된 느낌을 줍니다.
   이미지 태그: 셔츠, 타이다이, 패션, 컬러풀, 의류, 스타일, 디자인, 옷걸이

3. ID: PROD004
   이름: 러닝 운동화
   브랜드: 스피드런
   가격: 149,000원
   이미지 캡션: 빨간 배경에 놓인 운동화.
   이미지 설명: 이미지는 밝은 빨간색 배경 위에 놓인 한 켤레의 주황색 및 빨간색 계열의 운동화를 보여줍니다. 운동화는 깔끔한 디자인과 눈길을 사로잡는 흰색 로고가 특징입니다.
   이미지 태그: 운동화, 신발, 빨간색, 형광, 스포츠, 로고, 패션, 스타일



In [9]:
##############################################
# 6-2. 특정 제품 상세 조회
##############################################
# 첫 번째 제품 ID로 상세 조회
if documents:
    first_doc_id = documents[0]["id"]
    detailed_result = search_client.get_document(key=first_doc_id)
    
    print(f"🔍 제품 상세 정보 (ID: {first_doc_id})")
    print("="*80)
    print(f"이름: {detailed_result['name']}")
    print(f"브랜드: {detailed_result['brand']}")
    print(f"가격: {detailed_result['price']:,.0f}원")
    print(f"설명: {detailed_result['description']}")
    print(f"\n이미지 캡션: {detailed_result.get('imageCaption', 'N/A')}")
    print(f"이미지 설명: {detailed_result.get('imageDescription', 'N/A')}")
    print(f"이미지 태그: {', '.join(detailed_result.get('imageTags', []))}")
    print(f"\n이미지 URL: {detailed_result['imageUrl']}")
    print(f"벡터 차원: {len(detailed_result['descriptionVector'])}개")
    print("="*80)

🔍 제품 상세 정보 (ID: PROD001)
이름: 클래식 화이트 티셔츠
브랜드: 베이직코튼
가격: 29,000원
설명: 100% 순면으로 제작된 기본 화이트 티셔츠입니다. 데일리룩으로 완벽하며 편안한 착용감을 제공합니다.

이미지 캡션: 흰색 티셔츠를 입은 사람.
이미지 설명: 이미지에는 심플하고 기본적인 흰색 반팔 티셔츠를 입은 사람이 등장하며, 배경은 식물이 있는 실내 공간입니다.
이미지 태그: 티셔츠, 흰색, 반팔, 심플, 식물, 실내

이미지 URL: https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?w=800&q=80
벡터 차원: 3072개
